# Video Anomaly detection

The automation of detecting anomalous events in videos is a challenging problem that currently attracts a lot of attention by researchers, but also has broad applications across industry verticals.  

The approach involves training deep neural networks to develop an in-depth understanding of the physical and causal rules in the observed scenes. The model effectively learns to predict future frames in the video. By calculating the error in this prediction, it is then possible to detect if something unusual, an anomalous event, occurred, if there is a large prediction error.  

The approach can be used both in a supervised and unsupervised fashion, thus enabling the detection of pre-defined anomalies, but also of anomalous events that have never occurred in the past. 

> Post on [LinkedIn](https://www.linkedin.com/feed/update/urn:li:activity:6512538611181846528) (includes video demonstration)

# Learning Goals

You will learn:
1. How to adapt an existing neural network architecture to your use-case.
1. How to prepare video data for deep learning. 
1. How to perform hyperparameter tuning with [HyperDrive](https://azure.microsoft.com/en-us/blog/experimentation-using-azure-machine-learning/) to improve the performance of you model.
1. How to deploy a deep neural network as a webservice for video processing. 
1. How to post-process the output of a Keras model for secondary tasks (here, anomaly detection)
2. How to define a build pipeline for DevOps.


In [3]:
%%HTML
<video width="800" controls>
  <source src="anomaly.mp4" type="video/mp4">
</video>

## Connect to Azure Machine Learning workspace and prepare compute

In [ ]:
ws = Workspace.from_config(path=config_json, auth=svc_pr)


# choose a name for your cluster
gpu_compute_name = config['gpu_compute']

try:
    gpu_compute_target = AmlCompute(workspace=ws, name=gpu_compute_name)
    print("found existing compute target: %s" % gpu_compute_name)
except ComputeTargetException:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                                max_nodes=5,
                                                                idle_seconds_before_scaledown=1800)

    # create the cluster
    gpu_compute_target = ComputeTarget.create(ws, gpu_compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    gpu_compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

## Train Tensorflow model

In [ ]:
est = TensorFlow(source_directory=script_folder,
                compute_target=gpu_compute_target,
                pip_packages=['keras==2.0.8', 'theano', 'tensorflow==1.8.0', 'tensorflow-gpu==1.8.0', 'matplotlib', 'horovod', 'hickle'],
                entry_script='train.py', 
                use_gpu=True,
                node_count=1,
                script_params={"--remote_execution": None, "--data-folder": "https://coursematerial.blob.core.windows.net/ucsd-ad/"}
                )
                
experiment_name = "prednet_train"

exp = Experiment(ws, experiment_name)

run = exp.submit(est)

run.wait_for_completion(show_output=True)